In [1]:
import os
os.chdir('/deca')   

In [2]:
import zipfile
import numpy as np
from pathlib import Path

# ----------------------------
# Utility functions
# ----------------------------

def list_npz_contents(path):
    """Return list of internal .npy filenames inside .npz"""
    with zipfile.ZipFile(path, 'r') as z:
        return z.namelist()

def npy_header_from_npz(npz_path, array_name):
    """Read array metadata (shape, dtype) without loading data."""
    with zipfile.ZipFile(npz_path) as z:
        with z.open(array_name) as f:
            version = np.lib.format.read_magic(f)
            shape, fortran, dtype = np.lib.format._read_array_header(f, version)
            return shape, dtype, fortran

def array_sizes(path):
    """Return compressed storage sizes of each array inside .npz"""
    with zipfile.ZipFile(path, 'r') as z:
        return {name: z.getinfo(name).file_size for name in z.namelist()}


# ----------------------------
# Paths to your files
# ----------------------------
albedo_path = Path("data/FLAME_albedo_from_BFM.npz")
texture_path = Path("data/FLAME_texture.npz")

print("=== Comparing NPZ Contents ===")
a_keys = list_npz_contents(albedo_path)
t_keys = list_npz_contents(texture_path)

print("Albedo keys :", a_keys)
print("Texture keys:", t_keys)

common = set(a_keys).intersection(t_keys)
print("\nCommon keys:", common)

print("\n=== Header Comparison ===")
for key in common:
    meta_a = npy_header_from_npz(albedo_path, key)
    meta_t = npy_header_from_npz(texture_path, key)
    print(f"{key}:")
    print("  albedo :", meta_a)
    print("  texture:", meta_t)
    print()

print("\n=== Size Comparison ===")
print("Albedo sizes :", array_sizes(albedo_path))
print("Texture sizes:", array_sizes(texture_path))


=== Comparing NPZ Contents ===
Albedo keys : ['MU.npy', 'PC.npy']
Texture keys: ['vt.npy', 'ft.npy', 'tex_dir.npy', 'mean.npy']

Common keys: set()

=== Header Comparison ===

=== Size Comparison ===
Albedo sizes : {'MU.npy': 6291584, 'PC.npy': 1251999872}
Texture sizes: {'vt.npy': 82016, 'ft.npy': 119840, 'tex_dir.npy': 1258291328, 'mean.npy': 6291584}
